In [ ]:
import pickle
import numpy as np
import heapq
import os
import pandas as pd
import requests
from PIL import Image
from io import BytesIO

folder_path = 'vectores'

# Cargar los vectores
archivos_pkl = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]

embeddings = []
image_names = []

for archivo in archivos_pkl:
    archivo_pkl = os.path.join(folder_path, archivo)
    
    with open(archivo_pkl, 'rb') as f:
        datos = pickle.load(f)
    
    for item in datos:
        if 'embedding' in item and item['embedding'] is not None and len(item['embedding']) > 0:
            embeddings.append(item['embedding'][0])  
            image_names.append(item['image_name'])  

def KnnSearchLineal(data, query, k):
    max_heap = []
    
    for i in range(len(data)):
        distance = np.linalg.norm(data[i] - query)  
        
        if len(max_heap) < k:
            heapq.heappush(max_heap, (-distance, i))  
        else:
            heapq.heappushpop(max_heap, (-distance, i))
    
    # Devolver las imágenes y sus distancias
    return [(image_names[i], -dist) for dist, i in max_heap]  




In [ ]:
images_df = pd.read_csv('images.csv/images.csv')

# Establecer la consulta
query_index = 6
query = embeddings[query_index]  
imagen_q_name = image_names[query_index]

# Obtener el nombre de la imagen de la query
imagen_q = images_df[images_df['filename'] == imagen_q_name].iloc[0]

print("La imagen de la query es: ", imagen_q)

# Link de la imagen de la query
image_link = imagen_q['link']
print("Link de la imagen:", image_link)

# Realizar la búsqueda KNN
k = 5
resultados = KnnSearchLineal(embeddings, query, k)
print(f"Las {k} imágenes más cercanas a {image_names[query_index]} son:")

for img_name, dist in resultados:
    print(f"Imagen: {img_name}, Distancia: {dist}")
    image_row = images_df[images_df['filename'] == img_name]
    if not image_row.empty:
        print("Link de la imagen:", image_row['link'].values[0])


La imagen de la query es:  filename                                            30805.jpg
link        http://assets.myntassets.com/v1/images/style/p...
Name: 6, dtype: object
Link de la imagen: http://assets.myntassets.com/v1/images/style/properties/06e9d4231254fdb2c7fe967ad413ad7b_images.jpg
Las 5 imágenes más cercanas a 30805.jpg son:
Imagen: 11398.jpg, Distancia: 8.606139361605164
Link de la imagen: http://assets.myntassets.com/v1/images/style/properties/322a7416320456a75df831775b16981c_images.jpg
Imagen: 17751.jpg, Distancia: 8.191418148559112
Link de la imagen: http://assets.myntassets.com/v1/images/style/properties/6e03fb7365a228ef94e46ac09d25d1c8_images.jpg
Imagen: 14669.jpg, Distancia: 8.555687516962067
Link de la imagen: http://assets.myntassets.com/v1/images/style/properties/3aa18746c06db000ce9177998ef34f2b_images.jpg
Imagen: 30805.jpg, Distancia: 0.0
Link de la imagen: http://assets.myntassets.com/v1/images/style/properties/06e9d4231254fdb2c7fe967ad413ad7b_images.jpg
Imagen: 